In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/boostcamp/NER

/content/drive/MyDrive/boostcamp/NER


In [1]:
#!pip install torch==1.6.0
!pip install transformers==3.3.1
!pip install seqeval
!pip install fastprogress
!pip install attrdict

     |████████████████████████████████| 1.1 MB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 4.5 MB/s eta 0:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.7.0
    Uninstalling tokenizers-0.7.0:
      Successfully uninstalled tokenizers-0.7.0
  Attempting uninstall: transformers
    Found existing installation: transformers 2.11.0
    Uninstalling transformers-2.11.0:
      Successfully uninstalled transformers-2.11.0
     |████████████████████████████████| 43 kB 388 kB/s eta 0:00:011
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16170 sha256=6da09abf39fb18f3578645d6d11659508cf2711d59167344e80dc848255f4bcf
  Stored in directory: /Users/jinseok/Library/Caches/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


해당 과제의 코드는 박장원님의 "https://github.com/monologg/KoELECTRA/tree/master/finetune" 레포지토리의 코드를 변경하여 만들어졌습니다.

추신. 좋은 코드 만들어주시고, 코드 사용을 흔쾌히 허락해주신 박장원님께 감사인사 드립니다.

학습을 시작하기 전에 같이 동봉된 "preprocessing.ipynb"를 실행하여 나오는 "train.tsv","dev.tsv"를 "data/custom-ner"에 넣어줍니다.

In [8]:
!python3 run_ner.py --task custom-ner --config_file koelectra-small.json # config/custom-ner 에서 원하는 모델의 config를 변경하고 넣어주세요.

02/20/2021 02:23:01 - INFO - filelock -   Lock 140538074622096 acquired on /Users/jinseok/.cache/torch/transformers/8da255e6abb41f960d45d41af1e9efda71eb73e1fbbd1091b63b443888a1c3a3.73a6db14d6ede1dd8840a057bd410929cf4f97dc456644ca8b80b76f09166942.lock
Downloading: 100%|██████████████████████████████| 466/466 [00:00<00:00, 191kB/s]
02/20/2021 02:23:02 - INFO - filelock -   Lock 140538074622096 released on /Users/jinseok/.cache/torch/transformers/8da255e6abb41f960d45d41af1e9efda71eb73e1fbbd1091b63b443888a1c3a3.73a6db14d6ede1dd8840a057bd410929cf4f97dc456644ca8b80b76f09166942.lock
02/20/2021 02:23:03 - INFO - filelock -   Lock 140538074619664 acquired on /Users/jinseok/.cache/torch/transformers/825effcd21ded11fe1adfedb39968a0a60354b9c2e6bc769c36543f7c5f3c541.79565c1f8213906e95b08997d8e3f41e4b5ef049ffab0e770dfe3dea3dce562d.lock
Downloading: 100%|████████████████████████████| 279k/279k [00:00<00:00, 355kB/s]
02/20/2021 02:23:04 - INFO - filelock -   Lock 140538074619664 released on /Users/jin

In [ ]:
asdads

In [9]:
from transformers import ElectraTokenizer, ElectraForTokenClassification  #다른 모델을 쓸 때는 tokenizer(ex.ElectraTokenizer)와 model(ex.ElectraForTokenClassification)를 바꿔주어야 합니다.

  
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")  #tokenizer를 바꾸었다면 from_pretrained에 들어가는 부분을 바꿔주어야 합니다. model config에 있는 model 주소를 적어주시면 됩니다. ex. monologg/koelectra-base-v3-discriminator

model = ElectraForTokenClassification.from_pretrained("./ckpt/koelectra-small-custom-ner-ckpt/checkpoint-5000")


In [10]:
label_list=['DT-B','DT-I','LC-B','LC-I','OG-B','OG-I','PS-B','PS-I','QT-B','QT-I','TI-B','TI-I','O']

Kaggle Competition page에서 "test.txt"를 다운 받아서 "data/custom-ner"에 넣어줍니다.

In [11]:
texts=[]
with open("data/custom-ner/test.txt") as f:
    for line in f:
        texts.append(line.rsplit("\n")[0])

In [12]:
len(texts), texts[:5]

(7000,
 ['인터넷언론 이데일리 출신인 이숙현 안랩 커뮤니케이션팀 부장은 부대변인에 인선됐다',
  '엔제리너스의 이벤트 소식이 온라인을 달구고 있는 가운데 트위터로 전해진 다양한 드립을 모아봤다',
  '사건 직후 A 씨는 매장 직원의 신고로 출동한 경찰관에 의해 현지 숙소에서 신분 확인을 위한 조사를 받은 것으로 알려졌다',
  '경찰에 따르면 조씨 등은 지난해 1월부터 최근까지 서울 용산강남중구의 오피스텔아파트 등지에서 외국인 남성 관광객이 2030대 남성과 성관계 또는 유사성행위를 하도록 주선하고 그 대가로 10만20만원씩 총 6억여원을 챙긴 혐의를 받고 있다',
  '경찰에 따르면 A 씨는 24일 밤 10시 21분경 군산시내 한 음식점 앞에서 17살 B 군을 흉기로 한 차례 찔러 살해한 혐의다'])

In [13]:
import torch
from tqdm.auto import tqdm

In [14]:
labels=[]
for idx, sequence in tqdm(enumerate(texts)):

    tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
    inputs = tokenizer.encode(sequence, return_tensors="pt")
    outputs = model(inputs)
    predictions = torch.argmax(outputs[0], dim=2)
    labels_=[]
    for i,j in zip(tokens,predictions[0].tolist()):
        if i in ['[CLS]', '[SEP]']:
            continue
        if "##" in i:
            continue
        labels_.append(label_list[j])
    labels.extend(labels_)  

IndexError: index out of range in self

"submission.csv"를 kaggle competition에 제출합니다.

데이터 전처리,모델 config 튜닝을 이용하여 더 좋은 성적을 얻어보세요.

In [ ]:
with open("submission.csv","w") as f:
  f.write("id,tag\n")
  for idx,tag in enumerate(labels):
    f.write(str(idx))
    f.write(",")
    f.write(tag)
    f.write("\n")